<style>
    h1 {
        padding: 8px 8px;
        background-image: linear-gradient(135deg, #c9f3ff, rgb(131, 218, 255));
        font-weight: 700;
        position: static;
        text-align: center;
        color: #006098;
        font-size: 36px;
    }
    h2 {
        font-weight: 700;
        text-align: center;
        font-style: italic;
        font-size: 24px;
    }
</style>

<div><h1>XÂY DỰNG MODEL</h1></div>
<div><h2>SVM model via SVR</h2></div>

# Khai báo thư viện cần thiết

In [2]:
import pandas as pd
import polars as pl
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import re
# ! pip install nltk
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
# ! pip install pyspellchecker
from spellchecker import SpellChecker
# ! pip install spacy
from spacy import load
from urllib.parse import urlparse

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.svm import SVR
from sklearn.metrics import cohen_kappa_score

/home/dagngyen/miniconda3/envs/min_ds-env/lib/python3.10/site-packages/seaborn/_statistics.py:32: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.22.1)
  from scipy.stats import gaussian_kde
<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


# Đọc dữ liệu

- Đọc dữ liệu trên Google Drive.

In [2]:
# from google.colab import drive
# drive.mount('/gdrive', force_remount=True)

In [3]:
# train = pd.read_csv('/gdrive/MyDrive/Colab Notebooks/data/train.csv')
# train.sample(5)

In [4]:
# test = pd.read_csv('/gdrive/MyDrive/Colab Notebooks/data/test.csv')
# test

- Đọc dữ liệu trên local path.

In [5]:
train = pd.read_csv('./learning-agency-lab-automated-essay-scoring-2/train.csv')
train.sample(5)

,essay_id,full_text,score
1035,1061742,There are millions of wrecks in the U.S. every...,4
12836,bd0afc4,I think that other people should join this Sea...,2
2564,2688e0d,I feel the elctoral college is overpowered and...,1
11534,a984775,We should keep the Electoral College its a goo...,2
8413,7d09ab7,I am all for Thomas Huang new invation. I thin...,2


In [6]:
test = pd.read_csv('./learning-agency-lab-automated-essay-scoring-2/test.csv')
test.head()

,essay_id,full_text
0,000d118,Many people have car where they live. The thin...
1,000fe60,I am a scientist at NASA that is discussing th...
2,001ab80,People always wish they had the same technolog...


In [7]:
# Remove score from train and concate with test
all_data = pd.concat([train.iloc[:,:-1], test], axis=0).reset_index(drop=True)
all_data.sample(5)

,essay_id,full_text
13360,c41bf97,Many people think the face on mars was created...
16926,fa29121,"Hello my name is Mr. Baker with NASA, and we a..."
11291,a6526c3,Freedom without a car\n\nAround the world in d...
16508,f389a4c,"Throughout the years, technology has improved ..."
16589,f4ab05d,I whole heartedly believe driverless cars are ...


In [8]:
all_data.shape

(17310, 2)

# Xử lý dữ liệu

Ta sẽ loại bỏ các ký tự dư thừa và chuỗi không cung cấp nhiều ý nghĩa thông tin.

In [9]:
def remove_excess_str(text):
    # Loại bỏ các username bắt đầu @
    text = re.sub("@\w+", '', text)
    # Loại bỏ các thẻ HTML
    html = re.compile(r'<.*?>')
    text = html.sub(r'', text)
    # Loại bỏ URL
    text = [word for word in text.split() if not urlparse(word).scheme]
    text = ' '.join(text)
    # Loại bỏ dấu nháy đơn mà theo sau nó là chữ số
    text = re.sub("'\d+", "", text)
    # Loại bỏ các ký tự dư thừa như khoảng trắng, dấu chấm, dấu phẩy
    text = re.sub(r"\.+", ".", text)
    text = re.sub(r"\,+", ",", text)
    text = ' '.join(text.split())
    return text

# Trích lọc đặc trưng từ dữ liệu

Đầu tiên, ta định nghĩa và cài đặt hàm đếm số lượng từ vựng bị sử dụng sai.

In [10]:
def count_spelling_errors(word_list):
    return len(list(SpellChecker().unknown(word_list)))

Tiếp theo, định nghĩa hàm loại bỏ dấu câu để thống kê thông số về từ.

In [11]:
import string
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

Tiếp theo, cài đặt hàm phân tách thành các đoạn văn, thành các câu để thống kê.

In [12]:
def split_paragraphs(text):
    new_text = re.split('\n\n', text)
    return new_text

Cài đặt hàm trích lọc những từ khóa đóng góp quan trọng trong ý nghĩa của câu.

In [13]:
# Load Spacy model
nlp = load("en_core_web_sm")

# Define stop words
stop_words = set(stopwords.words('english'))

# Preprocess the text
def filter_word(text):
    text = text.lower()
    words = word_tokenize(text)
    words = [word for word in words if word.isalnum() and word not in stop_words]
    return words

Phần này, ta sẽ quan sát những giá trị thống kê từ văn bản:
- Tổng số lượng đoạn văn;
- Thống kê theo đoạn văn: Sau khi phân tách văn bản và xử lý dữ liệu, ta sẽ thống kê:
    + Số lượng câu: Giá trị nhỏ nhất, tứ phân vị thứ nhất, trung vị, tứ phân vị thứ ba, giá trị lớn nhất;
    + Số lượng từ: Giá trị nhỏ nhất, tứ phân vị thứ nhất, trung vị, tứ phân vị thứ ba, giá trị lớn nhất;
- Tổng số lượng câu;
- Thống kê theo câu: Sau khi phân tách thành các câu và xử lý dữ liệu, ta sẽ thống kê:
    + Số lượng từ: Giá trị nhỏ nhất, tứ phân vị thứ nhất, trung vị, tứ phân vị thứ ba, giá trị lớn nhất;
- Tổng số lượng từ;
- Tổng số lượng từ sau khi trích lọc những từ đóng góp ý nghĩa cho câu, đoạn văn;
- Thống kê theo từ: Trích lọc những từ đóng góp ý nghĩa cho câu, đoạn văn:
    + Số lượng ký tự: Giá trị nhỏ nhất, tứ phân vị thứ nhất, trung vị, tứ phân vị thứ ba, giá trị lớn nhất;
- Số lượng từ loại sai chính tả trong đoạn văn.

In [14]:
def paragraph_engineering(paragraph):
    _paragraph = list(map(remove_excess_str, paragraph))
    _lst_sents = []
    _lst_words = []
    for para in _paragraph:
        _lst_sents.append(len(sent_tokenize(para)))
        _lst_words.append(len(word_tokenize(para)))
    return len(_paragraph),\
            np.min(_lst_sents), np.quantile(_lst_sents, 0.25), np.median(_lst_sents), np.quantile(_lst_sents, 0.75), np.max(_lst_sents),\
            np.min(_lst_words), np.quantile(_lst_words, 0.25), np.median(_lst_words), np.quantile(_lst_words, 0.75), np.max(_lst_words)

In [15]:
all_data['num_para'],\
    all_data['para_min_sent'], all_data['para_q1_sent'], all_data['para_median_sent'], all_data['para_q3_sent'], all_data['para_max_sent'],\
    all_data['para_min_word'], all_data['para_q1_word'], all_data['para_median_word'], all_data['para_q3_word'], all_data['para_max_word'] = zip(*all_data['full_text'].map(split_paragraphs).map(paragraph_engineering))

In [16]:
def sentence_engineering(sentence):
    _lst_words = []
    for sent in sentence:
        _lst_words.append(len(word_tokenize(sent)))
    return len(sentence),\
            np.min(_lst_words), np.quantile(_lst_words, 0.25), np.median(_lst_words), np.quantile(_lst_words, 0.75), np.max(_lst_words)

In [17]:
all_data['num_sent'],\
    all_data['sent_min_word'], all_data['sent_q1_word'], all_data['sent_median_word'], all_data['sent_q3_word'], all_data['sent_max_word']\
        = zip(*all_data['full_text'].apply(remove_excess_str).map(sent_tokenize).map(sentence_engineering))

In [18]:
def word_engineering(word):
    _lst_chars = list(map(len, word))
    return len(word), np.min(_lst_chars), np.quantile(_lst_chars, 0.25), np.median(_lst_chars), np.quantile(_lst_chars, 0.75), np.max(_lst_chars)

In [19]:
word_full_text = all_data['full_text'].map(remove_excess_str).map(remove_punctuation)
word_full_text_no_stopwords = word_full_text.map(filter_word)
# all_data['num_error'] = word_full_text.map(word_tokenize).map(count_spelling_errors)
all_data['num_word'] = word_full_text.map(word_tokenize).map(len)
all_data['num_word_no_stopwords'], all_data['word_min_chars'], all_data['word_q1_chars'], all_data['word_median_chars'], all_data['word_q3_chars'], all_data['word_max_chars'] = zip(*word_full_text_no_stopwords.map(word_engineering))

Số lượng đặc trung sau khi rút trích thông tin từ dữ liệu.

In [20]:
all_data.shape

(17310, 26)

# Trích lọc đặc trưng thông qua TF-IDF

"TF-IDF là viết tắt của “Term Frequency, Inverse Document Frequency” - tạm dịch “Tần suất thuật ngữ, Tần suất tài liệu nghịch đảo”. Đó là một cách để chấm điểm tầm quan trọng của các từ (hoặc \"các thuật ngữ\") dựa trên tần suất xuất hiện của chúng xuất hiện trên nhiều tài liệu dựa trên quy tắc sau:"
- Nếu một từ xuất hiện thường xuyên trong tài liệu, điều đó rất quan trọng. Cho từ này điểm cao.
- Nhưng nếu một từ xuất hiện trong nhiều tài liệu, thì đó không phải là mã định danh duy nhất. Cho từ đó điểm thấp.

Do đó, những từ phổ biến như `the` và `for` xuất hiện trong nhiều tài liệu sẽ được scaled down. Các từ xuất hiện thường xuyên trong một tài liệu sẽ được scaled up.

Với những giải thích trên, ta có công thức tính trọng số của một từ trong tài liệu trong ngữ liệu như sau:
$$w_{i,j} = tf_{i,j} \cdot idf_i = tf_{i,j} \cdot log(\frac {N}{df_i})$$

Trong đó:
- $tf_{i,j}$: Tần suất xuất hiện của i trong j
- $N$: Tổng số tài liệu
- $df_i$: Số tài liệu chứa i

**Reference:** https://medium.com/analytics-vidhya/an-introduction-to-tf-idf-using-python-5f9d1a343f77"

In [21]:
# Khởi tạo TfidfVectorizer với các tham số cụ thể
vectorizer = TfidfVectorizer(
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    token_pattern=None,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(1, 3),
    min_df=0.05,
    max_df=0.95,
    sublinear_tf=True,
)

# Sử dụng TfidfVectorizer cho dữ liệu
tfidf = vectorizer.fit_transform([row for row in all_data['full_text']]).toarray()

# Lưu kết quả
colums_tfidf = [f'tfidf_{i}' for i in range(tfidf.shape[1])]
tfidf_df = pd.DataFrame(tfidf, columns=colums_tfidf)
all_data = pd.concat([all_data, tfidf_df], axis=1)

In [22]:
all_data

,essay_id,full_text,num_para,para_min_sent,para_q1_sent,para_median_sent,para_q3_sent,para_max_sent,para_min_word,para_q1_word,...,tfidf_3282,tfidf_3283,tfidf_3284,tfidf_3285,tfidf_3286,tfidf_3287,tfidf_3288,tfidf_3289,tfidf_3290,tfidf_3291
0,000d118,Many people have car where they live. The thin...,1,13,13.00,13.0,13.0,13,545,545.00,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.034733,0.071065,0.0,0.0
1,000fe60,I am a scientist at NASA that is discussing th...,5,2,3.00,3.0,5.0,8,44,52.00,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2,001ab80,People always wish they had the same technolog...,4,4,4.00,5.5,7.5,9,92,104.75,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
3,001bdc0,"We all heard about Venus, the planet without a...",5,2,2.00,4.0,6.0,7,25,69.00,...,0.0,0.043294,0.057518,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",6,1,1.25,3.0,4.0,4,3,22.00,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17305,fffb49b,"In ""The Challenge of Exporing Venus,"" the auth...",1,11,11.00,11.0,11.0,11,264,264.00,...,0.0,0.023939,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
17306,fffed3e,Venus is worthy place to study but dangerous. ...,4,1,1.00,1.5,3.5,8,4,16.00,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
17307,000d118,Many people have car where they live. The thin...,1,13,13.00,13.0,13.0,13,545,545.00,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.034733,0.071065,0.0,0.0
17308,000fe60,I am a scientist at NASA that is discussing th...,5,2,3.00,3.0,5.0,8,44,52.00,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0


# Xây dựng mô hình: SVM

## _Chuẩn bị dữ liệu_

In [23]:
# Extract train and test from all_data
train_processed = all_data.iloc[:train.shape[0], :].copy()
train_processed['score'] = train['score']
test_processed = all_data.iloc[train.shape[0]:, :].copy()
test_processed.reset_index(drop=True, inplace=True)

In [24]:
target_col = ['score']
drop_cols = ['essay_id', 'full_text']

## _Xây dựng kiểm tra chéo dữ liệu (cross validation) bằng StratifiedKFold_

In [26]:
# Build StratifiedKFold with 5 folds
FOLDS = 2
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)
for fold, (train_index, val_index) in enumerate(skf.split(train_processed, train_processed['score'])):
    train_processed.loc[val_index, 'fold'] = fold

## _Xây dựng độ đo: Quadratic Weighted Kappa_

In [27]:
# Build Quadratic Weighted Kappa metric for evaluation SVM model
def qwk_score(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred.clip(1, 6).round(0), weights='quadratic')

## _Huấn luyện dữ liệu với StratifiedKFold và đưa ra dự đoán_

In [30]:
train_preds
test_preds

array([[0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32)

In [33]:
test_processed.columns

Index(['essay_id', 'full_text', 'num_para', 'para_min_sent', 'para_q1_sent',
       'para_median_sent', 'para_q3_sent', 'para_max_sent', 'para_min_word',
       'para_q1_word',
       ...
       'tfidf_3282', 'tfidf_3283', 'tfidf_3284', 'tfidf_3285', 'tfidf_3286',
       'tfidf_3287', 'tfidf_3288', 'tfidf_3289', 'tfidf_3290', 'tfidf_3291'],
      dtype='object', length=3318)

In [34]:
train_processed.drop(columns=target_col).columns

Index(['essay_id', 'full_text', 'num_para', 'para_min_sent', 'para_q1_sent',
       'para_median_sent', 'para_q3_sent', 'para_max_sent', 'para_min_word',
       'para_q1_word',
       ...
       'tfidf_3283', 'tfidf_3284', 'tfidf_3285', 'tfidf_3286', 'tfidf_3287',
       'tfidf_3288', 'tfidf_3289', 'tfidf_3290', 'tfidf_3291', 'fold'],
      dtype='object', length=3319)

In [40]:
temp=np.array([])
temp = np.append(temp, [model.predict(X_test)])
temp

array([2.90450383, 2.84732938, 4.07502989])

In [41]:
train_preds = np.zeros(len(train_processed), dtype='float32')
test_preds = np.zeros((len(test_processed),FOLDS), dtype='float32')

for fold in range(FOLDS):
    print('#'*25)
    print('### Fold',fold+1)
    print('#'*25)
    
    train_index = train_processed["fold"] != fold
    valid_index = train_processed["fold"] == fold
    
    X_train = train_processed[train_index].drop(columns=drop_cols+target_col+['fold'])
    y_train = train_processed[train_index][target_col]
    X_valid = train_processed[valid_index].drop(columns=drop_cols+target_col+['fold'])
    y_valid = train_processed[valid_index][target_col]
    X_test = test_processed.drop(columns=drop_cols)
    
    model = SVR(C=10)
    model.fit(X_train, y_train)
    train_preds[valid_index] = model.predict(X_valid)
    test_preds[:,fold] = model.predict(X_test)

    score = qwk_score(y_valid, train_preds[valid_index])    
    print(f"QWK score: {score}")
    print()
    
print('#'*25)
score = qwk_score(train_processed.score.values, train_preds)
print('Overall CV QWK score =',score)

#########################
### Fold 1
#########################


/home/dagngyen/miniconda3/envs/min_ds-env/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


QWK score: 0.6773548633062687

#########################
### Fold 2
#########################


/home/dagngyen/miniconda3/envs/min_ds-env/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


QWK score: 0.684604062265085

#########################
Overall CV QWK score = 0.6810279418953384


In [3]:
pd.cut([1.2, 3.6, 4.2, 5.4, 1], [-np.inf] + [1.5, 2.5, 3.5, 4.5, 5.5] + [np.inf], 
                    labels=[1,2,3,4,5,6]).astype('int32')

array([1, 4, 4, 5, 1], dtype=int32)

## _Xác định ngưỡng để làm tròn giá trị dự đoán của bài toán hồi quy_

Sử dụng mô hình hồi quy giúp cho việc học dữ liệu không bị overfitting. Vì thế vấn đề xảy ra khi sử dụng bài toán hồi quy cho biến dự đoán phân loại có thứ tự là xác định đúng được làm tròn số ở ngưỡng nào. Ví dụ, giá trị dự đoán là $1.6$, vậy ta sẽ chọn $1$ hay $2$? Ở bài toán này, chúng ta có 6 labels từ 1 đến 6, cho nên chúng ta sẽ có 5 ngưỡng cần xác định để làm tròn số. Chúng ta sử dụng QWK để làm độ đo tìm ra ngưỡng làm tròn.

In [5]:
def find_thresholds(true, pred, interrupt=50):
    # Khởi tạo ngưỡng ban đầu
    threshold = [1.5, 2.5, 3.5, 4.5, 5.5]
    pred_round = pd.cut(pred, [-np.inf] + threshold + [np.inf],
                        labels=[1,2,3,4,5,6]).astype('int32')
    best = cohen_kappa_score(true, pred_round, weights="quadratic")

    # Tìm lần lượt 5 ngưỡng
    for k in range(5):
        for sign in [1,-1]:
            v = threshold[k]
            threshold_test = threshold.copy()
            stop = 0
            while stop < interrupt:
                # Tìm giá trị ngưỡng mới
                v += sign * 0.01
                threshold_test[k] = v
                pred_round = pd.cut(pred, [-np.inf] + threshold_test + [np.inf],
                                    labels=[1,2,3,4,5,6]).astype('int32')
                metric = cohen_kappa_score(true, pred_round, weights="quadratic")

                # Phát hiện dừng sớm vòng lặp
                if metric <= best:
                    stop += 1
                else:
                    stop = 0
                    best = metric
                    threshold = threshold_test.copy()

    # Trả kết quả ngưỡng tốt nhất
    pred_round = pd.cut(pred, [-np.inf] + threshold + [np.inf], 
                    labels=[1,2,3,4,5,6]).astype('int32')
    return threshold, cohen_kappa_score(true, pred_round, weights="quadratic")

In [ ]:
best, thresholds = find_thresholds(train_processed.score.values, train_preds)